DATA BRICKS ASSESSMENT

NAME: RAMIREDDY PREETHI

BATCH: PYTHON BATCH 2

In [0]:
configs = {
    "fs.azure.account.key.hexa.blob.core.windows.net": "22tryO+X5649syvEBLfnKMWXxuLmZycQ1ZjRjYMM5y3qDtymyxvJqE5+44j2MUamDcnL0p89OPcZ+AStreKivQ=="
}

# all above configurations are used to create mount point
dbutils.fs.mount(
  source = "wasbs://orders@hexa.blob.core.windows.net/",
  mount_point = "/mnt/orders",
  extra_configs = configs)

True

In [0]:
df = spark.read.csv("/mnt/orders/orders.csv", header=True, inferSchema=True)
df.show()

+--------+-----------+-------------------+------+---------+
|order_id|customer_id|         order_date|amount|   status|
+--------+-----------+-------------------+------+---------+
|       1|        101|2023-01-15 10:30:00| 250.0|Completed|
|       2|        102|2023-02-20 14:00:00| 150.0|  Pending|
|       3|        103|2023-02-22 16:45:00| 200.0|Completed|
|       4|        101|2023-03-10 09:15:00| 100.0| Canceled|
|       5|        104|2023-03-12 11:00:00| 300.0|Completed|
|       6|        102|2023-04-05 13:30:00| 120.0|  Pending|
|       7|        105|2023-04-06 15:00:00| 180.0|Completed|
|       8|        103|2023-04-15 08:45:00| 220.0| Canceled|
+--------+-----------+-------------------+------+---------+



1. How would you optimize a query to calculate the frequency of order_id in the orders table?  



In [0]:
%sql
SELECT order_id, COUNT(*) AS frequency
FROM orders
GROUP BY order_id
ORDER BY frequency DESC;


order_id,frequency
1,1
6,1
3,1
5,1
4,1
8,1
7,1
2,1


2. How would you read a CSV file and write it as a Delta  in Databricks?

In [0]:
df = spark.read.csv("/mnt/orders/orders.csv", header=True, inferSchema=True)

# Write the DataFrame to Delta format
df.write.format("delta").mode("overwrite").save("/mnt/delta/orders")

In [0]:
%sql
create database if not exists orders;

In [0]:
df.write.format("delta").mode("overwrite").saveAsTable("default.orders")

In [0]:
%sql
select * from orders

order_id,customer_id,order_date,amount,status
1,101,2023-01-15T10:30:00Z,250.0,Completed
2,102,2023-02-20T14:00:00Z,150.0,Pending
3,103,2023-02-22T16:45:00Z,200.0,Completed
4,101,2023-03-10T09:15:00Z,100.0,Canceled
5,104,2023-03-12T11:00:00Z,300.0,Completed
6,102,2023-04-05T13:30:00Z,120.0,Pending
7,105,2023-04-06T15:00:00Z,180.0,Completed
8,103,2023-04-15T08:45:00Z,220.0,Canceled


3. How would you upsert (merge) new data into the `orders` Delta table?

In [0]:
%sql
CREATE TABLE IF NOT EXISTS default.new_orders (
    order_id integer,
    customer_id integer,
    order_date TIMESTAMP,
    amount DOUBLE,
    status STRING
);


In [0]:
%sql
INSERT INTO default.new_orders VALUES
(123, 101, '2024-12-01 10:15:30', 150.75, 'Pending'),
(124, 102, '2024-12-02 12:45:00', 250.00, 'Shipped'),
(125, 103, '2024-12-03 09:30:45', 99.99, 'Delivered'),
(126, 110, '2024-12-04 14:20:10', 300.50, 'Cancelled'),
(127, 115, '2024-12-05 16:00:00', 120.00, 'Pending'),
(128, 106, '2024-12-06 11:30:05', 450.25, 'Shipped'),
(129, 107, '2024-12-07 18:05:20', 80.00, 'Delivered');


num_affected_rows,num_inserted_rows
7,7


In [0]:
%sql 
SELECT * FROM default.new_orders;


order_id,customer_id,order_date,amount,status
123,101,2024-12-01T10:15:30Z,150.75,Pending
124,102,2024-12-02T12:45:00Z,250.0,Shipped
125,103,2024-12-03T09:30:45Z,99.99,Delivered
126,110,2024-12-04T14:20:10Z,300.5,Cancelled
127,115,2024-12-05T16:00:00Z,120.0,Pending
128,106,2024-12-06T11:30:05Z,450.25,Shipped
129,107,2024-12-07T18:05:20Z,80.0,Delivered


In [0]:
%sql
MERGE INTO default.orders AS target
USING default.new_orders AS source
ON target.order_id = source.order_id
WHEN MATCHED THEN
    UPDATE SET 
        target.customer_id = source.customer_id,
        target.amount = source.amount,
        target.order_date = source.order_date,
        target.status = source.status
WHEN NOT MATCHED THEN
    INSERT (order_id, customer_id, amount, order_date, status)
    VALUES (source.order_id, source.customer_id, source.amount, source.order_date, source.status);


num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
7,0,0,7


To check whether data is merged or not

In [0]:
%sql
SELECT * FROM default.orders WHERE order_id = 123;


order_id,customer_id,order_date,amount,status
123,101,2024-12-01T10:15:30Z,150.75,Pending


4. How would you perform time travel on the `orders` Delta table? 

In [0]:
%sql
SELECT * FROM default.orders VERSION AS OF 0;



order_id,customer_id,order_date,amount,status
1,101,2023-01-15T10:30:00Z,250.0,Completed
2,102,2023-02-20T14:00:00Z,150.0,Pending
3,103,2023-02-22T16:45:00Z,200.0,Completed
4,101,2023-03-10T09:15:00Z,100.0,Canceled
5,104,2023-03-12T11:00:00Z,300.0,Completed
6,102,2023-04-05T13:30:00Z,120.0,Pending
7,105,2023-04-06T15:00:00Z,180.0,Completed
8,103,2023-04-15T08:45:00Z,220.0,Canceled


In [0]:
%sql
SELECT * FROM default.orders VERSION AS OF 1;

order_id,customer_id,order_date,amount,status
1,101,2023-01-15T10:30:00Z,250.0,Completed
2,102,2023-02-20T14:00:00Z,150.0,Pending
3,103,2023-02-22T16:45:00Z,200.0,Completed
4,101,2023-03-10T09:15:00Z,100.0,Canceled
5,104,2023-03-12T11:00:00Z,300.0,Completed
6,102,2023-04-05T13:30:00Z,120.0,Pending
7,105,2023-04-06T15:00:00Z,180.0,Completed
8,103,2023-04-15T08:45:00Z,220.0,Canceled
123,101,2024-12-01T10:15:30Z,150.75,Pending
124,102,2024-12-02T12:45:00Z,250.0,Shipped


5. How would you delete older versions of the Delta table that are more than 7 days old?  


In [0]:
%sql
VACUUM default.orders RETAIN 168 HOURS;


path
dbfs:/user/hive/warehouse/orders
